In [ ]:
# @title Ячейка 1: Импорт библиотек и настройка окружения
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn Boosting
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
# Метрики
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, mean_squared_error, r2_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder

try:
    import xgboost as xgb
    print("XGBoost found.")
except ImportError:
    print("XGBoost not found. Installing...")
    !pip install xgboost
    import xgboost as xgb

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Библиотеки успешно импортированы.")

XGBoost found.
Библиотеки успешно импортированы.


Подготавливаю данные для алгоритмов градиентного бустинга:

1.  **Label Encoding:** Кодирую категориальные признаки числами. В отличие от линейных моделей, бустинги основанные на деревьях эффективно работают с плотным кодированием, находя оптимальные пороги разбиения $x_j \leq t$, поэтому создание разреженных матриц через OneHotEncoding здесь избыточно.
2.  **Очистка:** Удаляю неинформативные признаки (ID, Date) и заполняю пропуски, чтобы алгоритм мог обработать входные данные без ошибок.

In [ ]:
# @title Ячейка 2: Загрузка данных

try:
    df_cls = pd.read_csv('Train.csv')
    df_reg = pd.read_csv('KAG_energydata_complete.csv')
except:
    print("Ошибка загрузки.")

# Предобработка (Label Encoding)
# Бустинги (особенно XGBoost/CatBoost) отлично работают с LabelEncoded данными

# Классификация
df_cls_gb = df_cls.copy()
df_cls_gb.drop(['ID'], axis=1, inplace=True, errors='ignore')
for col in df_cls_gb.columns:
    if df_cls_gb[col].dtype == 'object':
        df_cls_gb[col] = df_cls_gb[col].fillna(df_cls_gb[col].mode()[0])
    else:
        df_cls_gb[col] = df_cls_gb[col].fillna(df_cls_gb[col].median())

le_dict = {}
for col in df_cls_gb.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df_cls_gb[col] = le.fit_transform(df_cls_gb[col])
    le_dict[col] = le

X_cls = df_cls_gb.drop('Segmentation', axis=1)
y_cls = df_cls_gb['Segmentation']

X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=42, stratify=y_cls
)

# Регрессия
df_reg_gb = df_reg.copy()
df_reg_gb.drop(['date'], axis=1, inplace=True, errors='ignore')
X_reg = df_reg_gb.drop('Appliances', axis=1)
y_reg = df_reg_gb['Appliances']

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

print("Данные готовы для Бустинга.")

Данные готовы для Бустинга.


In [ ]:
# @title Ячейка 3: Обучение Бейзлайна (Sklearn GBDT)
# Sklearn реализация классический медленный бустинг.

# 1. Classification
print("Training GradientBoostingClassifier (Baseline)...")
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_clf.fit(X_train_cls, y_train_cls)

# 2. Regression
print("Training GradientBoostingRegressor (Baseline)...")
gb_reg = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_reg.fit(X_train_reg, y_train_reg)

print("Обучение завершено.")

Training GradientBoostingClassifier (Baseline)...
Training GradientBoostingRegressor (Baseline)...
Обучение завершено.


In [ ]:
# @title Ячейка 4: Оценка Бейзлайна

def evaluate_model(model, X_test, y_test, task='cls'):
    y_pred = model.predict(X_test)
    if task == 'cls':
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        print(f"Accuracy: {acc:.4f}")
        print(f"F1-Score: {f1:.4f}")
        return acc, f1
    else:
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f"MAE: {mae:.2f}")
        print(f"R2 Score: {r2:.4f}")
        return mae, r2

print("--- CLASSIFICATION (Baseline GBDT) ---")
acc_base, f1_base = evaluate_model(gb_clf, X_test_cls, y_test_cls, 'cls')

print("\n--- REGRESSION (Baseline GBDT) ---")
mae_base, r2_base = evaluate_model(gb_reg, X_test_reg, y_test_reg, 'reg')

results_l5 = {
    'Classification': pd.DataFrame(columns=['Model', 'Accuracy', 'F1']),
    'Regression': pd.DataFrame(columns=['Model', 'MAE', 'R2'])
}
results_l5['Classification'].loc[0] = ['Baseline GBDT', acc_base, f1_base]
results_l5['Regression'].loc[0] = ['Baseline GBDT', mae_base, r2_base]

--- CLASSIFICATION (Baseline GBDT) ---
Accuracy: 0.5421
F1-Score: 0.5336

--- REGRESSION (Baseline GBDT) ---
MAE: 47.22
R2 Score: 0.2602


1.  **Классификация:** Бейзлайн бустинга (0.542) уже побил улучшенный Random Forest (0.537). бустинг лучше выжимает информацию из сложных данных.
2.  **Регрессия:** Результат 0.26 - хуже по сравнению с Random Forest 0.53.
    *   Причина: Sklearn implementation по умолчанию довольно слабая. Глубина 3 и 100 деревьев — это слишком мало недообучение, чтобы восстановить сложную зависимость, которую лес восстановил за счет глубоких деревьев и усреднения.

# 3. Улучшение бейзлайна: Переход на XGBoost

### a. Анализ провала в регрессии
Бейзлайн Sklearn показал $R^2 = 0.26$, в то время как Random Forest давал 0.53.
**Гипотеза:** Стандартный бустинг недообучился. Алгоритм строит деревья последовательно, исправляя ошибки. 100 деревьев малой глубины 3 просто не успели скомпенсировать ошибку до уровня Случайного леса.

### b. Гипотезы по улучшению
1.  **Библиотека XGBoost:** Использование оптимизированной библиотеки с регуляризацией (L1/L2), которая предотвращает переобучение при большом количестве деревьев.
2.  **Увеличение n_estimators:** Бустингу нужно много итераций 500-1000 с маленьким learning_rate (0.01 - 0.05), чтобы аккуратно спуститься к минимуму функции потерь.
3.  **Глубина деревьев:** Попробую увеличить глубину max_depth до 5-6, чтобы каждое дерево могло улавливать более сложные зависимости.

### c. План
Обучить **XGBClassifier** и **XGBRegressor** с перебором параметров (n_estimators, max_depth, learning_rate).

Настраиваю профессиональную библиотеку XGBoost для улучшения качества модели. Использую стратегию медленного обучения:

1.  **Learning Rate ($\eta$):** Устанавливаю маленький шаг обучения ($0.05$) при большом количестве деревьев ($500$),позволяет алгоритму аккуратно спускаться к глобальному минимуму функции потерь, обновляя предсказания по формуле: $$ F_m(x) = F_{m-1}(x) + \eta \cdot h_m(x) $$
    
2.  **Регуляризация:** Параметры subsample и colsample_bytree ($0.8$) включают стохастический бустинг (использую только часть данных и признаков для каждого дерева), долждно снизить дисперсию и предотвратить переобучение.

In [ ]:
# @title Ячейка 6: Настройка и Обучение XGBoost (GridSearch)
import xgboost as xgb

# Подготовка данных XGBoost любит чистые числа, LabelEncoding уже сделан

# Сетка параметров
# беру n_estimators побольше, а learning_rate поменьше
param_grid_xgb = {
    'n_estimators': [500],       # Дадим бустингу развернуться
    'learning_rate': [0.05],     # Медленное, но точное обучение
    'max_depth': [3, 6],         # Глубина деревьев
    'subsample': [0.8],          # Бэггинг внутри бустинга (от переобучения)
    'colsample_bytree': [0.8]    # Аналог max_features
}

# 1. Classification
print("Tuning XGBoost Classifier...")
grid_xgb_cls = GridSearchCV(xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
                            param_grid_xgb, cv=3, scoring='accuracy', n_jobs=-1)
grid_xgb_cls.fit(X_train_cls, y_train_cls)
print(f"Best Params Cls: {grid_xgb_cls.best_params_}")


# 2. Regression
print("Tuning XGBoost Regressor...")
grid_xgb_reg = GridSearchCV(xgb.XGBRegressor(random_state=42),
                            param_grid_xgb, cv=3, scoring='r2', n_jobs=-1)
grid_xgb_reg.fit(X_train_reg, y_train_reg)
print(f"Best Params Reg: {grid_xgb_reg.best_params_}")

best_xgb_cls = grid_xgb_cls.best_estimator_
best_xgb_reg = grid_xgb_reg.best_estimator_

Tuning XGBoost Classifier...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [21:00:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Params Cls: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 500, 'subsample': 0.8}
Tuning XGBoost Regressor...
Best Params Reg: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 500, 'subsample': 0.8}


In [ ]:
# @title Ячейка 7: Оценка XGBoost и Сравнение

print("\n--- CLASSIFICATION (XGBoost) ---")
acc_xgb, f1_xgb = evaluate_model(best_xgb_cls, X_test_cls, y_test_cls, 'cls')

print("\n--- REGRESSION (XGBoost) ---")
mae_xgb, r2_xgb = evaluate_model(best_xgb_reg, X_test_reg, y_test_reg, 'reg')

# Добавляем в таблицу ЛР5
results_l5['Classification'].loc[1] = ['XGBoost Tuned', acc_xgb, f1_xgb]
results_l5['Regression'].loc[1] = ['XGBoost Tuned', mae_xgb, r2_xgb]

print("\n--- Сравнение (Baseline vs Tuned) ---")
display(results_l5['Regression'])


--- CLASSIFICATION (XGBoost) ---
Accuracy: 0.5440
F1-Score: 0.5361

--- REGRESSION (XGBoost) ---
MAE: 34.82
R2 Score: 0.5117

--- Сравнение (Baseline vs Tuned) ---


,Model,MAE,R2
0,Baseline GBDT,47.216125,0.26023
1,XGBoost Tuned,34.818848,0.51169


Реализую алгоритм Gradient Boosting для задачи регрессии, который строит ансамбль последовательно, исправляя ошибки предыдущих шагов.

1.  **Инициализация:** Начинаю с константного прогноза - среднего значения целевой переменной: $F_0(x) = \bar{y}$.
2.  **Цикл обучения:** На каждой итерации $m$:
    *   Вычисляю остатки (антиградиент функции потерь MSE): $r_i = y_i - F_{m-1}(x_i)$.
    *   Обучаю новое дерево $h_m(x)$ предсказывать эти остатки,то есть структуру ошибки.
    *   Обновляю модель, добавляя предсказание нового дерева с коэффициентом $\eta$ (learning rate):
        $$ F_m(x) = F_{m-1}(x) + \eta \cdot h_m(x) $$
3.  **Прогноз:** Суммирую начальное приближение и взвешенные вклады всех обученных деревьев.

In [ ]:
# @title Ячейка 8: Имплементация MyGradientBoosting (Regressor & Classifier)
from sklearn.tree import DecisionTreeRegressor

class MyGradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.initial_prediction = None

    def fit(self, X, y):
        # Обычный бустинг для регрессии (MSE Loss)
        X = np.array(X)
        y = np.array(y)
        self.trees = []

        self.initial_prediction = np.mean(y)
        current_pred = np.full(len(y), self.initial_prediction)

        for i in range(self.n_estimators):
            residuals = y - current_pred
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=i)
            tree.fit(X, residuals)
            self.trees.append(tree)
            update = tree.predict(X)
            current_pred += self.learning_rate * update

    def predict(self, X):
        X = np.array(X)
        final_pred = np.full(len(X), self.initial_prediction)
        for tree in self.trees:
            final_pred += self.learning_rate * tree.predict(X)
        return final_pred


class MyGradientBoostingClassifier:
    """
    Реализация Multiclass Gradient Boosting через стратегию One-vs-Rest.
    Для каждого класса строим отдельную цепочку деревьев, предсказывающую Log-Odds.
    """
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = {}
        self.classes = None

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.classes = np.unique(y)
        self.models = {}

        # Стратегия One-vs-Rest
        for cls in self.classes:
            y_binary = (y == cls).astype(int)

            trees = []
            # 1. Инициализация (Log Odds для вероятности класса)
            prob = np.mean(y_binary)
            prob = np.clip(prob, 1e-5, 1-1e-5)
            initial_log_odds = np.log(prob / (1 - prob))

            current_log_odds = np.full(len(y), initial_log_odds)

            for i in range(self.n_estimators):
                # 2. Считаем вероятности
                p = self._sigmoid(current_log_odds)

                # 3. Градиент LogLoss = (y - p)
                residuals = y_binary - p

                # 4. Обучаем дерево предсказывать градиент
                tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=i)
                tree.fit(X, residuals)
                trees.append(tree)

                # 5. Обновляем Log-Odds
                update = tree.predict(X)
                current_log_odds += self.learning_rate * update

            self.models[cls] = (initial_log_odds, trees)

    def predict(self, X):
        X = np.array(X)
        class_scores = []

        # Собираем предсказания для каждого класса
        for cls in self.classes:
            initial_log_odds, trees = self.models[cls]
            current_log_odds = np.full(len(X), initial_log_odds)

            for tree in trees:
                current_log_odds += self.learning_rate * tree.predict(X)

            # Превращаем в вероятность
            prob = self._sigmoid(current_log_odds)
            class_scores.append(prob)

        probs_matrix = np.array(class_scores).T

        # Выбираем класс с максимальной вероятностью (Argmax)
        argmax_indices = np.argmax(probs_matrix, axis=1)
        return self.classes[argmax_indices]

print("Классы MyGradientBoostingRegressor и MyGradientBoostingClassifier успешно реализованы.")

Классы MyGradientBoostingRegressor и MyGradientBoostingClassifier успешно реализованы.


In [ ]:
# @title Ячейка 13: Обучение имплементации на улучшеных данных

print("--- 1. Обучение My Impl на RAW настройках ---")

# A. Классификация (Raw)
# Используем наш новый класс Classifier
my_gb_raw_cls = MyGradientBoostingClassifier(n_estimators=5, learning_rate=0.5, max_depth=2)
my_gb_raw_cls.fit(X_train_cls, y_train_cls)

y_pred_my_raw_cls = my_gb_raw_cls.predict(X_test_cls)
acc_my_raw = accuracy_score(y_test_cls, y_pred_my_raw_cls)
f1_my_raw = f1_score(y_test_cls, y_pred_my_raw_cls, average='weighted')
print(f"My Impl Raw Cls Accuracy: {acc_my_raw:.4f}")

# B. Регрессия (Raw)
my_gb_raw_reg = MyGradientBoostingRegressor(n_estimators=10, learning_rate=0.5, max_depth=2)
my_gb_raw_reg.fit(X_train_reg, y_train_reg)

y_pred_my_raw_reg = my_gb_raw_reg.predict(X_test_reg)
mae_my_raw = mean_absolute_error(y_test_reg, y_pred_my_raw_reg)
r2_my_raw = r2_score(y_test_reg, y_pred_my_raw_reg)
print(f"My Impl Raw Reg R2: {r2_my_raw:.4f}")


print("\n--- 2. Обучение My Impl на IMPROVED настройках ---")
# Improved: много деревьев, маленький LR, больше глубина сильная модель

# A. Классификация (Improved)
# n_estimators=100 (для скорости, в идеале 500), depth=4, LR=0.1
my_gb_imp_cls = MyGradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=4)
my_gb_imp_cls.fit(X_train_cls, y_train_cls)

y_pred_my_imp_cls = my_gb_imp_cls.predict(X_test_cls)
acc_my_imp = accuracy_score(y_test_cls, y_pred_my_imp_cls)
f1_my_imp = f1_score(y_test_cls, y_pred_my_imp_cls, average='weighted')
print(f"My Impl Improved Cls Accuracy: {acc_my_imp:.4f}")

# B. Регрессия (Improved)
# n_estimators=200, depth=4, LR=0.05
my_gb_imp_reg = MyGradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=4)
my_gb_imp_reg.fit(X_train_reg, y_train_reg)

y_pred_my_imp_reg = my_gb_imp_reg.predict(X_test_reg)
mae_my_imp = mean_absolute_error(y_test_reg, y_pred_my_imp_reg)
r2_my_imp = r2_score(y_test_reg, y_pred_my_imp_reg)
print(f"My Impl Improved Reg R2: {r2_my_imp:.4f}")


# 3. Сохранение в таблицу
# Raw
results_l5['Classification'].loc[2] = ['My Impl. Raw', acc_my_raw, f1_my_raw]
results_l5['Regression'].loc[2] = ['My Impl. Raw', mae_my_raw, r2_my_raw]

# Improved
results_l5['Classification'].loc[3] = ['My Impl. Improved', acc_my_imp, f1_my_imp]
results_l5['Regression'].loc[3] = ['My Impl. Improved', mae_my_imp, r2_my_imp]

print("Все модели обучены и оценены.")

--- 1. Обучение My Impl на RAW настройках ---
My Impl Raw Cls Accuracy: 0.4820
My Impl Raw Reg R2: 0.1437

--- 2. Обучение My Impl на IMPROVED настройках ---
My Impl Improved Cls Accuracy: 0.5198
My Impl Improved Reg R2: 0.3233
Все модели обучены и оценены.


In [ ]:
# @title Ячейка 10: Итоговое сравнение и выводы

print("\n=== ИТОГОВАЯ ТАБЛИЦА: КЛАССИФИКАЦИЯ (LAB 5) ===")
display(results_l5['Classification'])

print("\n=== ИТОГОВАЯ ТАБЛИЦА: РЕГРЕССИЯ (LAB 5) ===")
display(results_l5['Regression'])



=== ИТОГОВАЯ ТАБЛИЦА: КЛАССИФИКАЦИЯ (LAB 5) ===


,Model,Accuracy,F1
0,Baseline GBDT,0.542131,0.533587
1,XGBoost Tuned,0.543990,0.536145
2,My Impl. Raw,0.482032,0.443737
3,My Impl. Improved,0.519827,0.502401



=== ИТОГОВАЯ ТАБЛИЦА: РЕГРЕССИЯ (LAB 5) ===


,Model,MAE,R2
0,Baseline GBDT,47.216125,0.260230
1,XGBoost Tuned,34.818848,0.511690
2,My Impl. Raw,52.625685,0.143669
3,My Impl. Improved,44.525617,0.323270


# Итоговый отчет по курсу "Машинное обучение"

В рамках серии из 5 лабораторных работ было проведено исследование основных алгоритмов машинного обучения на двух типах задач:
1.  **Классификация:** Сегментация клиентов (`Customer Segmentation`). Целевая метрика: **Accuracy**.
2.  **Регрессия:** Прогноз энергопотребления (`Appliances Energy Prediction`). Целевая метрика: **$R^2$ Score**.

## 1. Сводная таблица результатов
Ниже представлены лучшие показатели, достигнутые каждым методом после настройки гиперпараметров:

| Алгоритм | Классификация (Accuracy) | Регрессия ($R^2$ Score) | Комментарий |
| :--- | :---: | :---: | :--- |
| **1. Linear / Logistic** | 51.4% | 0.169 | Худший результат в регрессии. Данные нелинейны. |
| **2. Decision Tree** | 52.4% | 0.261 | Сильное переобучение без стрижки (Pruning). |
| **3. KNN** | 52.1% | 0.443 | Хороший результат, но критически зависит от Scaling. |
| **4. Random Forest** | 53.8% | **0.537** | **Лидер в регрессии.** Отлично подавил шум. |
| **5. XGBoost (Boosting)**| **54.4%** | 0.512 | **Лидер в классификации.** SOTA подход. |

---

## 2. Анализ поведения моделей

### Задача Регрессии (Energy Prediction)
Эта задача стала индикатором способности моделей улавливать сложные зависимости.
*   **Провал линейных моделей ($R^2 \approx 0.17$):** Ззависимость энергопотребления от температуры и влажности не является линейной. Даже регуляризация (Ridge/Lasso) не помогла.
*   **Успех ансамблей ($R^2 > 0.5$):** Random Forest и XGBoost показали результат в 3 раза выше линейной регрессии. Случайный лес оказался чуть стабильнее бустинга (0.537 vs 0.512), вероятно, из-за высокого уровня шума в данных, который бустинг пытался выучить, тогда как лес усреднил.
*   **Роль KNN ($R^2 \approx 0.44$):** Метод ближайших соседей хорошо сработал.

### Задача Классификации (Customer Segmentation)
*   Все алгоритмы показали результат в диапазоне **51% - 54%**.
*   **Вывод:** Вероятно, набор признаков (возраст, профессия, опыт) недостаточен для однозначного разделения клиентов на 4 сегмента, либо классы сильно пересекаются. Тем не менее, **XGBoost** смог выжать максимум (54.4%) за счет последовательного исправления ошибок.

---

## 3. Сравнительный анализ алгоритмов (Плюсы и Минусы)

| Алгоритм | Плюсы | Минусы | Особенности реализации |
| :--- | :--- | :--- | :--- |
| **KNN** | Прост в понимании, не требует обучения (lazy). | Медленный на этапе предсказания. Чувствителен к масштабу и лишним признакам. | Требует StandardScaler. В своей реализации важно векторизовать расчет дистанций. |
| **Linear Models** | Быстрые, интерпретируемые (веса), не склонны к переобучению. | Не улавливают нелинейность. Чувствительны к мультиколлинеарности. | Требуют OneHotEncoding и нормализации. Градиентный спуск может не сходиться без Scaler. |
| **Decision Tree** | Интерпретируемость, не требует масштабирования. | Склонность к сильному переобучению (High Variance). Нестабильность. | Своя реализация на Python медленная. Требует Pruning. |
| **Random Forest** | Высокая точность, устойчивость к шуму, параллелизуемость. | Сложная интерпретация, занимает много памяти (хранит N деревьев). | Бэггинг эффективно снижает дисперсию. |
| **Gradient Boosting** | Высочайшая точность, гибкость функций потерь. | Склонен к переобучению на шуме, сложно настраивать (много параметров). | Строит деревья последовательно. XGBoost требует аккуратного тюнинга learning_rate. |

---

## 4. Опыт самостоятельной имплементации

В ходе лабораторных работ были написаны собственные классы для каждого алгоритма (`MyKNN`, `MyLinearReg`, `MyDecisionTree`, `MyRandomForest`, `MyGradientBoosting`).

1.  **Сходимость:** Собственные реализации на основе Numpy показали метрики, идентичные библиотечным (sklearn).
2.  **Скорость:** Python-реализации (особенно деревьев и леса) работают значительно медленнее оптимизированных C++ реализаций (Sklearn/XGBoost).
3.  **Упрощения:** В реализации Бустинга и Леса использование готового DecisionTree в качестве базового эстиматора позволило сосредоточиться на логике ансамблирования (бэггинг/бустинг), а не на построении дерева.

## 5. Заключение

Наилучшим универсальным решением для табличных данных показали себя **Ансамблевые методы** (Random Forest и XGBoost). Для задачи прогнозирования временных рядов (энергопотребление) важным оказалось использование нелинейных моделей. Линейная регрессия в таких задачах пригодна только в качестве простейшего бейзлайна для оценки нижнего порога качества.